In [1]:
import pandas as pd
import numpy as np 
# NLP libraries
import string 
import nltk 
from nltk.corpus import stopwords
import re 
import pickle

In [2]:
with open('model_pickle','rb') as file:
    mp = pickle.load(file)

In [3]:
#loading dataset
df=pd.read_csv("test.csv",encoding="utf-8")
df.shape

(31089, 5)

In [4]:
#first five rows
df.head()

,Unnamed: 0,id,tweet_text,tweet_created_at,tweet_intent
0,96364,1212764,15% off ðŸ§™â€â™‚ï¸ #Wizardia NFTs - it's a ...,2022-08-06T16:56:35.000Z,NaN
1,96365,1212770,@asksabir007 @dao_lm Don't miss the chance to ...,2022-08-06T16:56:33.000Z,NaN
2,96366,1212773,This zesty faucet from @_bitcoiner is making m...,2022-08-06T16:56:33.000Z,NaN
3,96367,1212804,15% off ðŸ§™â€â™‚ï¸ #Wizardia NFTs - it's a ...,2022-08-06T16:56:23.000Z,NaN
4,96368,1212776,"@Kavaboy_NFT @tanya_4d Message me, letâ€™s tal...",2022-08-06T16:56:32.000Z,NaN


In [5]:
#information about dataset - datattype, nul values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31089 entries, 0 to 31088
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        31089 non-null  int64  
 1   id                31089 non-null  int64  
 2   tweet_text        31089 non-null  object 
 3   tweet_created_at  31089 non-null  object 
 4   tweet_intent      0 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ MB


In [6]:
#data cleaning
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', '', text)
    text = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?¿>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('@$%€¤«»±•|¦†‡¬¨¢“”','',text)             
    return text
df['tweet_text'] = df['tweet_text'].apply(lambda x: clean_text(x))

In [7]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
df['tweet_text'] = df['tweet_text'].apply(lambda x: tokenizer.tokenize(x))

In [8]:
def remove_stopwords(text):
    """
    Removing stopwords belonging to english language
    
    """
    words = [w for w in text if w not in stopwords.words('english')]
    return words

df['tweet_text'] = df['tweet_text'].apply(lambda x : remove_stopwords(x))

In [9]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text
df['tweet_text'] = df['tweet_text'].apply(lambda x : combine_text(x))

In [10]:
a=mp.predict(df['tweet_text'])

In [11]:
b = pd.Series(a)

In [12]:
df['tweet_intent'] = b 
df.head()

,Unnamed: 0,id,tweet_text,tweet_created_at,tweet_intent
0,96364,1212764,ðÿ â â ï wizardia nfts game magicfilled metave...,2022-08-06T16:56:35.000Z,Whitelist
1,96365,1212770,daolm dont miss chance win nft worth simple st...,2022-08-06T16:56:33.000Z,Whitelist
2,96366,1212773,zesty faucet bitcoiner making tweet claim free...,2022-08-06T16:56:33.000Z,Whitelist
3,96367,1212804,ðÿ â â ï wizardia nfts game magicfilled metave...,2022-08-06T16:56:23.000Z,Whitelist
4,96368,1212776,kavaboynft message letâ talk marketing project,2022-08-06T16:56:32.000Z,Appreciation


In [13]:
df.to_csv("samplesubmission.csv", encoding = "utf-8")